In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Load the SST dataset
dataset = load_dataset("sst", "default")

# Access the training split of the dataset
train_dataset = dataset["train"]

# Extract features (X) and labels (y) from the training split
X = train_dataset["sentence"]
sentiment_scores = np.array(train_dataset["label"])

# Convert sentiment scores to discrete class labels using the mapping function
def mapping(a):
    if a <= 0.2:
        return "very negative"
    elif a <= 0.4:
        return 'negative'
    elif a <= 0.6:
        return 'neutral'
    elif a <= 0.8:
        return 'positive'
    elif a<=1 and a>0.8:
        return 'very positive'

y = np.array([mapping(score) for score in sentiment_scores])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with CountVectorizer and Multinomial Naive Bayes
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

# Predict the class for new instances
# new_sentences = ["I love this movie!", "This movie is terrible."]
# predicted_classes = model.predict(new_sentences)
# print("Predicted classes:", predicted_classes)


Accuracy: 0.3926272674078408
Predicted classes: ['positive' 'very negative']


In [30]:
import numpy as np
from datasets import load_dataset

# Load the SST dataset
dataset = load_dataset("sst", "default")

# Access the training split of the dataset
train_dataset = dataset["train"]

# Extract features (xs) and labels (ys) from the training split
sentence = train_dataset["sentence"]
labels = train_dataset["label"]

# Convert labels to numpy array if needed
labels = np.array(labels)

# Print the shapes to verify
print("Number of sentences in training dataset:", len(sentence))
print("Number of labels in training dataset:", len(labels))


def maping(a):
    if a <= 0.2:
        return "very negative"
    elif a <= 0.4:
        return 'negative'
    elif a <= 0.6:
        return 'neutral'
    elif a <= 0.8:
        return 'positive'
    elif a <= 1 and a > 0.8:
        return 'very positive'

# Apply mapping function to labels
classes = np.vectorize(maping)(labels)


def train_naive_bayes(D, C):
    # Calculate the number of documents in each class.
    class_counts = {c: 0 for c in C}
    for doc in D:
        class_counts[doc['class']] += 1
        # print(class_counts)
    # print(len(D))
    # Calculate the log-prior probabilities of each class.
    log_prior = {c: np.log(class_counts[c] / len(D)) for c in C}
    
    # for c in C:
    #     print(c, log_prior[c])
    #     print(np.log(class_counts[c] / len(D)))
        
    # Create a dictionary to store the word counts for each class.
    word_counts_by_class = {c: {} for c in C}

    # Create a vocabulary of all unique words in the training data.
    vocabulary = set()
    for doc in D:
        for word in doc['text'].split():
            # print(word)
            vocabulary.add(word)
            word_counts_by_class[doc['class']][word] = word_counts_by_class[doc['class']].get(word, 0) + 1  # .get(word,0)
    
    # print(word_counts_by_class)
    # Calculate the log-likelihood probabilities of each word given each class.
    log_likelihood = {}
    for word in vocabulary:
        log_likelihood[word] = {c: np.log((word_counts_by_class[c].get(word, 0) + 1) /
                                          (sum(word_counts_by_class[c].values()) + len(vocabulary)) + 1)
                                 for c in C}

    return log_prior, log_likelihood, vocabulary



def TestNaiveBayes(TestDoc, logPrior, log_likelihood, C, vocabulary):
    Sum = {c:logPrior[c]  for c in C}
    best_c = None
    max_Sum = -np.inf
    for c in C:
        Sum[c] = logPrior[c]
        for word in TestDoc:
            if word in vocabulary: # and c in log_likelihood.get(word, {}):  # Check if word exists for class c
                Sum[c] += log_likelihood[word][c]
                # print(Sum[c], log_likelihood[word][c])
        max_Sum = -np.inf         
        if Sum[c] > max_Sum:
            max_Sum = Sum[c]
            best_c = c
            print("max: ", max_Sum," best: ", best_c)
    # print(best_c)        
    return best_c


# Access the testing split of the dataset
test_dataset = dataset["test"]

# Extract features (xs) and labels (ys) from the testing split
test_sentences = test_dataset["sentence"]
test_labels = test_dataset["label"]

# Initialize variables for accuracy calculation
total_correct = 0
total_wrong = 0
total_samples = len(test_sentences)


# Prepare documents
documents = [{'text': s, 'class': c} for s, c in zip(sentence, classes)]


# Train the Naive Bayes classifier
log_prior, log_likelihood, vocabulary = train_naive_bayes(documents, set(classes))

# Convert vocabulary set to a list
vocabulary = list(vocabulary)
# Iterate over the test dataset and make predictions
for test_doc, true_class in zip(test_sentences, test_labels):
    predicted_class  = TestNaiveBayes(test_doc, log_prior, log_likelihood, set(classes), vocabulary)
    # print(predicted_class)
    # print(true_class)
    if predicted_class == maping(true_class):
        total_correct += 1
        print(f"Predicted: {predicted_class}, True: {maping(true_class)}, Document: {test_doc}")
    # else:
    #     print ("False prediction")
    #     print(f"Predicted: {predicted_class}, True: {maping(true_class)}, Document: {test_doc}")

# Calculate accuracy
accuracy = total_correct / total_samples
print("Accuracy:", accuracy)


Number of sentences in training dataset: 8544
Number of labels in training dataset: 8544
max:  -2.056453404503927  best:  very negative
max:  -1.6591777208239906  best:  neutral
max:  -1.3477684848769604  best:  negative
max:  -1.3021764124199702  best:  positive
max:  -1.891457092643172  best:  very positive
max:  -1.9226422628356588  best:  very negative
max:  -1.5039329379560125  best:  neutral
max:  -1.1725252705874467  best:  negative
max:  -1.1217382906064872  best:  positive
max:  -1.736522490881092  best:  very positive
max:  -1.9218770289102893  best:  very negative
max:  -1.5017397707260847  best:  neutral
max:  -1.1703570570763522  best:  negative
max:  -1.1196808393553936  best:  positive
max:  -1.7362636832946807  best:  very positive
Predicted: very positive, True: very positive, Document: Emerges as something rare , an issue movie that 's so honest and keenly observed that it does n't feel like one .
max:  -1.9462321733333963  best:  very negative
max:  -1.52901325274175

In [6]:
def confusion_matrix_impl(true_labels, predicted_labels, labels=None):
    if labels is None:
        labels = np.unique(np.concatenate((true_labels, predicted_labels)))
    num_labels = len(labels)
    cm = np.zeros((num_labels, num_labels), dtype=int)
    for i in range(len(true_labels)):
        true_idx = np.where(labels == true_labels[i])[0][0]
        pred_idx = np.where(labels == predicted_labels[i])[0][0]
        cm[true_idx, pred_idx] += 1
    return cm
confusionmatrix=confusion_matrix_impl(,)
confusionmatrix

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
import numpy as np

def precision_score_per_class(confusion_matrix):
    true_positives = np.diag(confusion_matrix)
    false_positives = np.sum(confusion_matrix, axis=0) - true_positives
    precision = true_positives / (true_positives + false_positives)
    return precision

def recall_score_per_class(confusion_matrix):
    true_positives = np.diag(confusion_matrix)
    false_negatives = np.sum(confusion_matrix, axis=1) - true_positives
    recall = true_positives / (true_positives + false_negatives)
    return recall

def f1_score_per_class(confusion_matrix):
    precision = precision_score_per_class(confusion_matrix)
    recall = recall_score_per_class(confusion_matrix)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def macro_averaged_precision(confusion_matrix):
    precision = precision_score_per_class(confusion_matrix)
    return np.mean(precision)

def macro_averaged_recall(confusion_matrix):
    recall = recall_score_per_class(confusion_matrix)
    return np.mean(recall)

def macro_averaged_f1_score(confusion_matrix):
    f1 = f1_score_per_class(confusion_matrix)
    return np.mean(f1)


print("Precision per class:", precision_score_per_class(confusion_matrix=confusionmatrix))
print("Recall per class:", recall_score_per_class(confusion_matrix=confusionmatrix))
print("F1 Score per class:", f1_score_per_class(confusion_matrix=confusionmatrix))

print("Macro-averaged Precision:", macro_averaged_precision(confusion_matrix=confusionmatrix))
print("Macro-averaged Recall:", macro_averaged_recall(confusion_matrix=confusionmatrix))
print("Macro-averaged F1 Score:", macro_averaged_f1_score(confusion_matrix=confusionmatrix))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
confusion_matrix(y_test_true,logistic_pred)

In [ ]:
precision_per_class_skitl = precision_score(y_test_true,logistic_pred, average=None)

recall_per_class_skitl = recall_score(y_test_true,logistic_pred, average=None)

f1_score_per_class_skitl = f1_score(y_test_true,logistic_pred, average=None)

macro_avg_precision_skitl = precision_score(y_test_true,logistic_pred, average='macro')

macro_avg_recall_skitl = recall_score(y_test_true,logistic_pred, average='macro')

macro_avg_f1_score_skitl = f1_score(y_test_true,logistic_pred, average='macro')

print("Precision per class:", precision_per_class_skitl)
print("Recall per class:",recall_per_class_skitl)
print("F1 Score per class:", f1_score_per_class_skitl)

print("Macro-averaged Precision:", macro_avg_precision_skitl)
print("Macro-averaged Recall:", macro_avg_recall_skitl)
print("Macro-averaged F1 Score:", macro_avg_f1_score_skitl)